In [147]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np
from sojo.pstabilizer import group_instructorss_by_qubits, char_to_index, Instructor
from sojo.pstabilizer import weightss_to_lambda, word_to_index, generate_pauli_combination, instructor_to_lut, index_to_word
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [149]:
num_qubits = 3
ins = Instructor(num_qubits)
for k in range(5):
	for i in range(num_qubits - 1):
		ins.append('cx', [i, i + 1])
	ins.append('cx', [num_qubits - 1, 0])
	for i in range(num_qubits):
		ins.append('rx', i, np.random.rand())
		ins.append('ry', i, np.random.rand())
		ins.append('rz', i, np.random.rand())

ins.clustering()

In [150]:
LUT = instructor_to_lut(ins)

(5, 3, 4, 4)


In [151]:
num_qubits = 3
term = ['zii']
k = 0
# Init
# 2**n is th
lambdas = np.ones(4**num_qubits)

weightss = []
for w, word in enumerate(term):
    weights = []
    for j, char in enumerate(word):
        index = char_to_index(char)
        weights.append(LUT[k][j][index])
    weightss.append(weights)
weightss = np.array(weightss)
lambdas = weightss_to_lambda(weightss, lambdas)

In [152]:
non_zero_indices = np.nonzero(lambdas)
non_zero_values = np.array(lambdas)[non_zero_indices]
non_zero_indices, non_zero_values
print(non_zero_indices)
print(non_zero_values)

(array([ 4,  8, 12]),)
[ 0.5326693  -0.34297665  0.77371211]


In [166]:

def map_cx(index, control, target, num_qubits):  
    word = list(index_to_word(index, num_qubits))
    char_control = word[control]
    char_target = word[target]
    lambdas = 1
    if char_control == 'i':
        if char_target == 'y':
            word[control] = 'z'
            word[target] = 'y'
        elif char_target == 'z':
            word[control] = 'z'
            word[target] = 'z'
    elif char_control == 'x':
        if char_target == 'i':
            word[control] = 'x'
            word[target] = 'x'
        elif char_target == 'x':
            word[control] = 'x'
            word[target] = 'i'
        elif char_target == 'y':
            word[control] = 'y'
            word[target] = 'z'
        elif char_target == 'z':
            lambdas = -1
            word[control] = 'y'
            word[target] = 'y'
    elif char_control == 'y':
        if char_target == 'i':
            word[control] = 'y'
            word[target] = 'x'
        elif char_target == 'x':
            word[control] = 'y'
            word[target] = 'y'
        elif char_target == 'y':
            word[control] = 'x'
            word[target] = 'z'
            lambdas = -1
        elif char_target == 'z':
            word[control] = 'x'
            word[target] = 'y'
    elif char_control == 'z':
        if char_target == 'y':
            word[control] = 'i'
            word[target] = 'y'
        elif char_target == 'z':
            word[control] = 'i'
            word[target] = 'z'
    return word_to_index(word)*lambdas

num_qubits = 2
for index in range(4**num_qubits):
    for control in range(num_qubits):
        for target in range(num_qubits):
            if control == target:
                continue
            else:
                print(control, target)      
                print(map_cx(index, 0, 1, num_qubits))


0 1
0
1 0
0
0 1
1
1 0
1
0 1
14
1 0
14
0 1
15
1 0
15
0 1
5
1 0
5
0 1
4
1 0
4
0 1
11
1 0
11
0 1
-10
1 0
-10
0 1
9
1 0
9
0 1
10
1 0
10
0 1
-7
1 0
-7
0 1
6
1 0
6
0 1
12
1 0
12
0 1
13
1 0
13
0 1
2
1 0
2
0 1
3
1 0
3


In [170]:

num_qubits = 4
indices = []
for control in range(num_qubits):
    for target in range(num_qubits):
        if control == target:
            continue
        else:
            indices.append([control, target])
print(indices)

[[0, 1], [0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]]
